In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
from IPython.display import display_markdown

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
"""
Assistant: The first image shows a green statue of the Statue of Liberty standing on a stone pedestal in front of a body of water. 
The statue is holding a torch in its right hand and a tablet in its left hand. The water is calm and there are no boats or other objects visible. 
The sky is clear and there are no clouds. The second image shows a bee on a pink flower. 
The bee is black and yellow and is collecting pollen from the flower. The flower is surrounded by green leaves.
"""


'\nAssistant: The first image shows a green statue of the Statue of Liberty standing on a stone pedestal in front of a body of water. \nThe statue is holding a torch in its right hand and a tablet in its left hand. The water is calm and there are no boats or other objects visible. \nThe sky is clear and there are no clouds. The second image shows a bee on a pink flower. \nThe bee is black and yellow and is collecting pollen from the flower. The flower is surrounded by green leaves.\n'

In [3]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Device name: NVIDIA GeForce RTX 4080 SUPER


In [26]:
# Load images
# image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
# image2 = load_image("https://blog.imglarger.com/content/images/2023/06/How-to-Get-a-JPG-File-with-High-Resolution-Tools-and-Tips.jpg")

image1 = Image.open("C:/Users/rbrul/Documents/GitHub/wip_test/SmolVLM/snowmount.jpg")
image2 = Image.open("C:/Users/rbrul/Documents/GitHub/wip_test/SmolVLM/waterhand.jpg")

In [5]:
# Initialize processor and model
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-Instruct",
#     torch_dtype=torch.bfloat16,
#     _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# ).to(DEVICE)

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.bfloat16,).to(DEVICE)


Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [27]:
# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images in 2 sentences with some detail?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

answer = generated_texts[0]
display_markdown(answer, raw=True)

User:








Can you describe the two images in 2 sentences with some detail?
Assistant: In the first image, a person is standing in the snow, while in the second image, a person's hand is touching the water.